In [1]:
module MyDual

import Base: +,-,*,/,==,!=,>,<,>=,<=
import Base: abs,conj,inv,zero,one,show
import LinearAlgebra: transpose,adjoint,norm

export Dual

struct Dual{T<:Number}
    a::T
    b::T
end

Dual(x::Number) = Dual(x, zero(T))

value(u::Dual)   = u.a
epsilon(u::Dual) = u.b

zero(::Type{MyDual.Dual}) = Dual(zero(T),zero(T))
one(::Type{MyDual.Dual})  = Dual(one(T),zero(T))

abs(u::Dual)  = abs(value(u))
norm(u::Dual) = norm(value(u))

+(u::Dual, v::Dual) = Dual(value(u) + value(v), epsilon(u) + epsilon(v))
-(u::Dual, v::Dual) = Dual(value(u) - value(v), epsilon(u) - epsilon(v))
*(u::Dual, v::Dual) = Dual(value(u)*value(v), epsilon(u)*value(v) + value(u)*epsilon(v))
/(u::Dual, v::Dual) = Dual(value(u)/value(v),(epsilon(u)*value(v) - value(u)*epsilon(v))/(value(v)*value(v)))

==(u::Dual, v::Dual) = norm(u) == norm(v)
!=(u::Dual, v::Dual) = norm(u) != norm(v)
>(u::Dual, v::Dual)  = norm(u) > norm(v)
>=(u::Dual, v::Dual) = norm(u) >= norm(v)
<(u::Dual, v::Dual)  = norm(u) < norm(v)
<=(u::Dual, v::Dual) = norm(u) <= norm(v)

+(x::Number, u::Dual) = Dual(value(u) + x, epsilon(u))
+(u::Dual, x::Number) = x + u

-(x::Number, u::Dual) = Dual(x - value(u), epsilon(u))
-(u::Dual, x::Number) = Dual(value(u) - x, epsilon(u))

*(x::Number, u::Dual) = Dual(x*value(u), x*epsilon(u))
*(u::Dual, x::Number) = x*u

/(u::Dual, x::Number) = (1.0/x)*u

conj(u::Dual)  = Dual(value(u),-epsilon(u))
inv(u::Dual)   = one(Dual)/u

transpose(u::Dual) = u
transpose(uu::Array{MyDual.Dual,2}) = [uu[j,i] for i=1:size(uu)[1],j=1:size(uu)[2]]
adjoint(u::MyDual.Dual) = u

convert(::Type{Dual}, x::Number) = Dual(x,zero(x))
promote_rule(::Type{Dual}, ::Type{<:Number}) = Dual

## show(io::IO,u::Dual) = print(io,value(u)," + (",epsilon(u),")ϵ")

function show(io::IO,u::Dual) 
 op::String = (epsilon(u) < 0.0) ? " - " : " + ";
 print(io,value(u),op,abs(epsilon(u))," ϵ")
end

end

Main.MyDual

In [2]:
using Main.MyDual

u1 = [Dual(randn(),randn()) for i = 1:25];
u2 = [Dual(randn(),randn()) for i = 1:25];

uu1 =  reshape(u1,5,5);
uu2 =  reshape(u2,5,5);

In [3]:
uu3 = uu1.*uu2

5×5 Array{Dual{Float64},2}:
 -0.0986598 + (-0.14015)ϵ  …  -0.939963 + (-0.689693)ϵ 
 0.403035 + (-1.56359)ϵ       0.204738 + (1.04758)ϵ    
 0.976677 + (1.73684)ϵ        -0.332949 + (-0.0354034)ϵ
 -1.75626 + (1.06143)ϵ        -1.2276 + (-2.59829)ϵ    
 -2.33537 + (-0.533074)ϵ      0.0632941 + (0.647912)ϵ  

In [4]:
uu4 = map((u)->u*u, uu3)

5×5 Array{Dual{Float64},2}:
 0.00973377 + (0.0276544)ϵ  …  0.88353 + (1.29657)ϵ     
 0.162438 + (-1.26036)ϵ        0.0419176 + (0.428957)ϵ  
 0.953898 + (3.39266)ϵ         0.110855 + (0.023575)ϵ   
 3.08446 + (-3.7283)ϵ          1.50701 + (6.37934)ϵ     
 5.45396 + (2.48985)ϵ          0.00400614 + (0.0820181)ϵ